# PHYLOGENETIC TREES WITH DIGITAL ANNEALER INSPIRED BY GRAPH SPLITTING

The main objective of this method is to consecutively divide a set of nodes or species with Fujitsu's Digital Annealer until a phylogenetic tree is obtained.

In [1]:
# imports related to the DA
from dadk.QUBOSolverCPU import *
from dadk.Solution_SolutionList import *
from dadk.BinPol import *

# other imports
from openpyxl import load_workbook
import csv
import numpy as np

## Data collecting

In order to be able to obtain a phylogenetic tree, we need a set of species and a measurement that allows us to determine how close they are to each other. This would constitute the nodes and the edges of the graph we are going to cut. 

For that, we have originally taken datasets from [Phylome DB](https://phylomedb.org/phylomes?s=expl), a database webpage of different phylomes that offers both the standard accepted phylogenetic tree and the amino acid sequences of the species. With this amino acid sequence, in FASTA format, we have used the $\verb|func_cleanfasta|$ to clean up the .fasta files and fed it to BLAST+. The program analyzes its database of aminoacid sequences and following a local comparison algorithm, returns several metrics, among which we find the bit scores. 

Bit scores are a good metric that take several characteristics into account to determine how closely related the two species are. They are not normalized when they first come out of BLAST+, so the first step to get a proper matrix of similarities is to normalize them. Let us see how we do that with the following code.

In [2]:
# IMPORT SIMILARITY MATRIX (EXCEL FILE)

def import_from_excel(filename):
    """
    Imports a similarity matrix from an Excel file and returns a list of nodes and a bitscore matrix.

    Args:
        filename (str): Path to the Excel file.

    Returns:
        nodes (list): List of unique node names.
        bitscore_matrix (numpy.ndarray): 2D numpy array representing the bitscore matrix.
    """

    # Open up the Excel file
    workbook = load_workbook(filename)
    
    # Get the first sheet
    worksheet = workbook.worksheets[0]
    
    column_list = []
    
    # Obtain a list of lists (each element is a column of the file)
    for c in worksheet.columns:
        row = [cell.value for cell in c]
        column_list.append(row)
    
    # Clean up header and select nodes_query column, nodes_sub column and bit_score column
    nodes_query = column_list[0][1:]
    nodes_sub = column_list[1][1:]
    bit_scores = column_list[11][1:]
    
    # Transform bitscores to float type
    bit_scores = [float(i) for i in bit_scores]

    # Run the subroutine that deletes duplicate nodes
    nodes = extract_nodes(nodes_query)

    # Run the subroutine that creates the matrix shape from the bitscore column
    bitscore_matrix = extract_matrix(nodes_query, nodes_sub, nodes, bit_scores)

    return(nodes, bitscore_matrix)

In [3]:
# IMPORT SIMILARITY MATRIX (TSV FILE)

def import_from_tsv(filename):

    """
    Imports a similarity matrix from a tsv file and returns a list of nodes and a bitscore matrix.

    Args:
        filename (str): Path to the Excel file.

    Returns:
        nodes (list): List of unique node names.
        bitscore_matrix (numpy.ndarray): 2D numpy array representing the bitscore matrix.
    """

    tsv_path = filename
    column_list = []
    
    # Reads all rows as lists first
    with open(tsv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        rows = list(reader)
    
    # Transpose rows for columns
    for col in zip(*rows):
        column_list.append(list(col))
    
    # Clean up header and select nodes_query column, nodes_sub column and bit_score column
    nodes_query = column_list[0]
    nodes_sub = column_list[1]
    bit_scores = column_list[11]
    
    # Transform bitscores to float type
    bit_scores = [float(i) for i in bit_scores]

    # Run the subroutine that deletes duplicate nodes
    nodes = extract_nodes(nodes_query)

    # Run the subroutine that creates the matrix shape from the bitscore column
    bitscore_matrix = extract_matrix(nodes_query, nodes_sub, nodes, bit_scores)

    return(nodes, bitscore_matrix)

The reason for $\verb|nodes_query|$ and $\verb|nodes_sub|$ to exist is that BLAST+ first creates a database with our .fasta file and calls the species in that database 'Subject'. Then it iteratively compares the sequences in the original .fasta ('Queries') with the ones on the database, so we have two columns with all the possible pairs in the set. To have a proper array with only the nodes (sequences) for future use, we will search through the first column and get the unique names out of it.

In [4]:
def extract_nodes(nodes_query):
    """
    Subroutine that extracts unique nodes from the nodes_query list.

    Args:
        nodes_query (list): Original query nodes list.

    Returns:
        nodes (list): List of unique node names.
    """

    nodes = []

    # we initialize the current node selected
    # the nodes are in order, so we just need to check when the name changes
    current_node_name = nodes_query[0]
    nodes.append(current_node_name)
    
    # we inspect all the nodes in nodes_query
    for i in range(len(nodes_query)):
        # select a new node only when the name changes
        if nodes_query[i] != current_node_name:
            nodes.append(nodes_query[i])
            # we update current_node_name
            current_node_name = nodes_query[i]
    
    return(nodes)

The bit scores are a measurement of how closely related the species are, so the bigger the score, the bigger the simmilarity. The bit score matrix will be, then, a simmilarity matrix.

Now, we will normalize the bit scores. We will use the formula

$$ Norm_{bit(i,j)} = \frac{bit(i,j)}{\mathrm{mean}\{bit(i,i), bit(j,j)\}}*100. $$



We must also rearrange the $\verb|bit_scores_norm|$ array into a matrix. This is not a trivial step, as the total length of the array is not exactly $nodes \times nodes $, since some species are so distantly related that BLAST+ has purged them out. For those cases, we will asign the default zero to the edge between those nodes.

In [5]:
def extract_matrix(nodes_query, nodes_sub, nodes, bit_scores):

    """
    Subroutine that builds the structure of the bitscore matrix from the nodes_query and 
    nodes_sub list and the values from the normalized the bit_scores list.

    Args:
        nodes_query (list): List of query nodes.
        nodes_sub (list): List of subject nodes.
        nodes (list): List of unique node names.
        bit_scores (list): List of bit scores.

    Returns:
        bitscore_matrix (numpy.ndarray): 2D numpy array representing the bitscore matrix.
    """

    n_total = len(bit_scores)
    
    bit_scores_norm = []
    
    # We iterate over the entire array of bit scores
    
    for k in range(n_total):
    
        # We save the name of the query and subject
        query_name = nodes_query[k]
        sub_name = nodes_sub[k]
    
        # We iterate over the rest of the list finding the bit(query,query) = bit(i,i)
        # and bit(subject,subject) = bit(j,j)
    
        for i in range(n_total):
            
            if nodes_query[i] == query_name and nodes_sub[i] == query_name:
                bit_i_i = bit_scores[i]
    
            if nodes_query[i] == sub_name and nodes_sub[i] == sub_name:
                bit_j_j = bit_scores[i]
    
        # Calculate the mean
        mean_scores = (bit_i_i + bit_j_j)/2
    
        # Apply normalization formula
        bit_scores_norm.append(bit_scores[k]*100/mean_scores)
        

    ### BIT SCORE MATRIX BUILD ###
    
    bitscore_matrix = np.zeros([len(nodes),len(nodes)])
    
    for k in range(n_total):
    
        # for each element in bit_scores_norm, we search for the corresponding
        # indexes (matching names) in the nodes array, thus we find the
        # corresponding column and row in the matrix
        
        index_query = nodes.index(nodes_query[k])
        index_sub = nodes.index(nodes_sub[k])
        bitscore_matrix[index_query,index_sub] = bit_scores_norm[k]


    ### TRIANGULATION ###

    # The matrix must be symmetrical and sometimes it is not symmetrical when it comes
    # directly out of the bitscore list values, so we force the symmetry by 
    # calculating the mean of the corresponding values    
    for i in range(len(nodes)):
        for j in range(len(nodes)):
            
            if bitscore_matrix[i][j] != bitscore_matrix[j][i]:
                # mean between element (i,j) and element (j,i)
                mean_score = (bitscore_matrix[i][j] + bitscore_matrix[j][i])/2
                # reset value of elements (i,j) and (j,i) to the new mean
                bitscore_matrix[i][j] = mean_score
                bitscore_matrix[j][i] = mean_score
    
            # DIAG ZEROS #
            # We make the diagonal values to be zero to avoid the creation of an edge 
            # between an element and itself
    
            if i == j:
                bitscore_matrix[i][j]=0

    return(bitscore_matrix)       

We find an interesting result when printing this matrix, which is that it is not exactly symmetrical. This is due to the algorithm that BLAST+ uses to compare queries against subjects. For example, if the amino acid sequence of a certain species is too short, it may yield different results when comparing that species with another as subject-query VS query-subject. In order to fix this, we have decided to triangulate the matrix by taking the mean between the corresponding entries $\mathrm{mean}(\verb|bitscore_matrix[i][j]|, \verb|bitscore_matrix[j][i]|)$.

## NMcutDA

In [6]:
# Simple function to hide the output of the QUBO solver

def _hide_pol_info(pol):
        pol.user_data['hide_scaling_info'] = True
        pol.user_data['hide_sampling_info'] = True
        return pol

### MinCut

We are looking to divide the graph of all possible connections (all-to-all) in a way that minimizes the relationships between groups of species. This is a classical MinCut problem,

$$ Min_{cut} = \sum_{i=1}^{n-1}\sum_{j=i+1}^n d_{ij}(x_i - x_j)^2, \qquad x_i=\{0,1\}. $$

However, if we repeatedly apply this method without any pelanization, we realize that the algorithm tends to isolate one node VS rest, resulting in pectinate trees. In order to avoid this, a balance penalization has been added to force the choosing of all possible combinations of nodes

$$ Min_{cut} = \sum_{i=1}^{n-1}\sum_{j=i+1}^n d_{ij}(x_i - x_j)^2 + \alpha \left( \sum_{i=1}^n x_i^1 - c \right)^2, $$

where $c$ is the number of nodes to separate in that iteration. For example, if we have $8$ nodes, we would want to have the combinations $1:7$, $2:6$, $3:5$ and $4:4$, so $c$ would take the values $c=\{1,2,3,4\}$, respectively. The DA is in charge of finding the minimun energy for each of those combinations, so we wouldn't need to analyze combinations like $5:3$ as it would yield the same energy as $3:5$.

### Ncut

As a way to ponder which cut $c:(n-c)$ is the best, a post-processing metric was implemented, called the Ncut. Ncut values, not only the similarity between the species within each node, but also the dissimilarities between the two clusters as a whole. Therefore, we would be avoiding the problem of singleing out a node each time. The formula for Ncut is the following:

$$ N_{cut} = \frac{Min_{cut}}{assoc(A,V)} + \frac{Min_{cut}}{assoc(B,V)} $$

with

$$ assoc(X,V) = \sum_{u \in X, t \in V} d_{ut}, \qquad X = \{A, B\}.$$

Once the minimum Ncut is chosen, that division is applied to the nodes and the process repeats again for each of the two new clusters.

In [7]:
def QUBO(nodes, bitscore_matrix, c, alpha):

    """
    Creates the QUBO polynomial for the NMCutDA problem.

    Args:
        nodes (list): List of unique nodes.
        bitscore_matrix (numpy.ndarray): 2D numpy array representing the bitscore matrix.
        c (int): Desired number of nodes in the first cluster.
        alpha (float): Weighting factor for the penalization term.

    Returns:
        nodes (list): List of unique node names.
        bitscore_matrix (numpy.ndarray): 2D numpy array representing the bitscore matrix.
    """

    # shape of the set of variables for this QUBO
    my_constant_bits = np.full((len(nodes),), -1, np.int8) 
    my_varshapeset = VarShapeSet(BitArrayShape(name='x', shape=(len(nodes),), axis_names = ['nodes'], constant_bits = my_constant_bits))
    # freeze the shape for all BinPols
    BinPol.freeze_var_shape_set(my_varshapeset)
    

    ### FIRST POLYNOM: MinCut ###
    
    # Open an empty polynomial with the frozen shape
    H_d = BinPol()

    # run for all possible values of i and j
    for i in range(len(nodes)-1):
        for j in range(i+1, len(nodes)):
            # open an auxiliary polynomial
            H_ij = BinPol()
            # create term (x_i - x_j)
            H_ij.add_term(1, ('x',i))
            H_ij.add_term(-1, ('x',j))
            # power of 2 (x_i - x_j)^2
            H_ij.power(2)
            # add coefficient d_ij
            bit_coef = bitscore_matrix[i][j].item() # change to float (DA doesnt like numpy.float) 
            H_ij = bit_coef*H_ij
            # add auxiliary polynomial to the total polynomial (sum)
            H_d.add(H_ij)

    
    ### SECOND POLYNOM: penalization ###
    
    # Auxiliary polynomial for sum(x_i)
    H_alpha_aux = BinPol()
    for i in range(len(nodes)):
        H_alpha_aux.add_term(1, ('x',i))
    # substract c and power of 2
    H_alpha = alpha*((H_alpha_aux-c).power(2))
    
    
    ### Total QUBO ###
    
    H_qubo = H_d + H_alpha
    H_qubo = _hide_pol_info(H_qubo)

    return(H_qubo)

In [8]:
### QUBO FUNCTION ITERATING C ###

def N_cut_func(nodes, bitscore_matrix, edges_times):

    """
    Selects the cut c:(n-c) with the lowest Ncut.

    Args:
        nodes (list): List of unique nodes.
        bitscore_matrix (numpy.ndarray): 2D numpy array representing the bitscore matrix.
        edges_times (str): 'single' if edges are counted once, 'double' if edges are counted twice.

    Returns:
        best_min_cut (float): Minimum Mincut value for the best partition.
        best_N_cut (float): Minimum Ncut value for the best partition.
        best_N_cut_array (numpy.ndarray): Array indicating the best partition of nodes.

    """

    # We choose alpha high enough that the DA respects the
    # value of c chosen.
    alpha = 1000

    # Options for the DA local Solver
    solver = QUBOSolverCPU(
        number_iterations = 200000,
        number_runs = 10,
        scaling_bit_precision = 32,
        scaling_action = ScalingAction.AUTO_SCALING)

    # We initialize N_cut to a high enough value
    N_cut = 10**50

    # Start of the loop, with c ranging from 1 to nodes/2 (all possible partitions)
    for c in range(1, int(len(nodes)/2)+1):
    
        # Create the QUBO with the corresponding c
        H_qubo = QUBO(nodes, bitscore_matrix, c, alpha)
    
        ### SOLVER ###
        
        solution_list = solver.minimize(H_qubo)
        solution = solution_list.min_solution

        # list of 0 and 1 indicating the partition
        min_cut_array = solution.configuration

        ### CHECK CONDITION ###

        # We check that the partition number c was met,
        # if not, we relaunch the solver with a higher alpha

        # Auxiliary counter to check how many times the condition c
        # was not met and to restore alpha to its original value
        counter = 0
        
        while sum(min_cut_array) != c:
            # update counter
            counter += 1
            # make alpha bigger
            alpha = alpha*10
            # create QUBO again with new alpha
            H_qubo = QUBO(nodes, bitscore_matrix, c, alpha)
            # new solution list
            solution_list = solver.minimize(H_qubo)
            solution = solution_list.min_solution
            min_cut_array = solution.configuration

        # restore original value of alpha
        alpha = alpha/(10**counter)
            
    
        ### N-CUT ###
        
        # compute the polynomial with the min_cut solution list to know 
        # the energy of the minimum cut
        min_cut = H_qubo.compute(min_cut_array)

        # We initialize assoc_A_V and assoc_B_V and calculate their values
    
        assoc_A_V = 0
        assoc_B_V = 0

        # It is not clear in the original assoc(A,V) formula if the edges should be counted 
        # once or twice. Both ways are implemented but 'twice' = 'double' was finally
        # selected for yielding better results

        ################################################### edges counted twice

        if edges_times == 'double':
        
            # index i refers to cluster A
            for i in range(len(nodes)):
                # index j runs over all nodes (V = A + B)
                for j in range(len(nodes)):
            
                    # sum of the edges associated to cluster A
                    if min_cut_array[i] == 0:
                        assoc_A_V += bitscore_matrix[i][j]
            
                    # sum of the edges associated to cluster B
                    elif min_cut_array[i] == 1:
                        assoc_B_V += bitscore_matrix[i][j]


        ################################################### edges counted once

        if edges_times == 'single':
        
            for i in range(len(nodes)):
                for j in range(len(nodes)):
                
                    if min_cut_array[i] == 0:
                        # add all edges between A and B
                        if min_cut_array[j] != 0:
                            assoc_A_V += bitscore_matrix[i][j]
                        # avoid edges related to nodes already added in A (i>=j)
                        elif i<j:
                            assoc_A_V += bitscore_matrix[i][j]
                
                    elif min_cut_array[i] == 1:
                        # add all edges between B and A
                        if min_cut_array[j] != 1:
                            assoc_B_V += bitscore_matrix[i][j]
                        # avoid edges related to nodes already added in B (i>=j)
                        elif i<j:
                            assoc_B_V += bitscore_matrix[i][j]

        ######################################################

        # Prevent extreme cases in which nodes dont have edges (edges = 0, thus divide by zero)

        if assoc_A_V == 0:
            assoc_A_V = 1
        if assoc_B_V == 0:
            assoc_B_V = 1

        # Formula for Ncut
    
        current_N_cut = min_cut/assoc_A_V + min_cut/assoc_B_V

        # We keep current_N_cut if it's lower than before (minimum)
    
        if current_N_cut < N_cut:
    
            best_min_cut = min_cut
            best_N_cut = current_N_cut
            best_N_cut_array = min_cut_array

            # Update old N_cut with current value
            
            N_cut = best_N_cut   

    return(best_min_cut, best_N_cut, best_N_cut_array)   

This function that we just saw $\verb|N_cut_func|$ chooses the best $c:(n-c)$ based on the Ncut value. It is now our turn to apply these changes and separate the node arrays iteratively. We are dealing with a binary split, so if we wanted to save the variables each time, they would grow exponentially. In order to optimize this, we have programmed a function that recursively calls itself until a final separation array is obtained.

The way to 'mark' the splits each time has been coded in binary. The first split determines a 'path' ($0$ if the DA has chosen $x_i = 0$ and $1$ if $x_i=1$). Equivalently, we can think of this method as a way to code if the branch goes 'up' ($0$) or 'down' ($1$), for example. That way, we can keep track of all the node splits and reconstruct the phylogenetic tree.

In [9]:
def assign_nodes(node_array, cut_array):

    """
    Function that assigns the split nodes to two new arrays based
    on the solution array of the DA.

    Args:
        node_array (list): original list of nodes with the species names
        cut_array (list): array of zeros and ones that indicates the separation of nodes in two groups

    Returns:
        nodes_0 (list): list of species names belonging to group A
        nodes_1 (list): list of species names belongin to group B

    """

    # We initialize empty lists
    nodes_0 = []
    nodes_1 = []
    
    for i in range(len(cut_array)):
    
        # based on the value of cut_array, separate nodes into the two lists
        if cut_array[i] == 0:
            nodes_0.append(node_array[i])
        elif cut_array[i] == 1:
            nodes_1.append(node_array[i])

    return(nodes_0, nodes_1)

In [10]:
def update_bit_matrix(old_bit_matrix, cut_array):

    """"
    Function that updates the bitscore matrix based on the solution array of the DA, 
    splitting the necessary columns and rows into two matrices in corcondance
    with cut_array

    Args:
        old_bit_matrix (numpy.ndarray): original bitscore matrix to split in two
        cut_array (list): solution of the NMCutDA problem, array consisting of zeros and ones

    Returns:
        bitscore_matrix_0 (numpy.ndarray): bitscore matrix associated with group A (nodes_0)
        bitscore_matrix_1 (numpy.ndarray): bitscore matrix associated with group B (nodes_1)

    """

    # initialized arrays for the split of indices of columns and rows related
    # to array nodes_0 and array nodes_1, respectively
    index_0 = []
    index_1 = []

    # loop that searches for the indices of the columns and rows
    # associated to zero (respectively, one)

    for i in range(len(cut_array)):

        if cut_array[i] == 0:
            index_0.append(i)

        elif cut_array[i] == 1:
            index_1.append(i)

    # deletes rows and columns associated with 1 to build bitscore_matrix_0
 
    bitscore_matrix_0 = np.delete(old_bit_matrix, index_1, axis=0)
    bitscore_matrix_0 = np.delete(bitscore_matrix_0, index_1, axis=1)

    # deletes rows and columns associated with 0 to build bitscore_matrix_1
    bitscore_matrix_1 = np.delete(old_bit_matrix, index_0, axis=0)
    bitscore_matrix_1 = np.delete(bitscore_matrix_1, index_0, axis=1)

    return(bitscore_matrix_0, bitscore_matrix_1)

In [11]:
def split_nodes(node_array, bitscore_matrix, edges_times, key='0', result=None):

    """ 
    Recursive function that takes an array of nodes and recursively splits them
    by calling the QUBO solver until there is only one node per group. Saves the result
    into a dictionary with a code '0010110' that indicates to which group the node has belonged
    throughout the process.

    Args:
        node_array (list): Initial list of nodes.
        bitscore_matrix (numpy.ndarray): initial bitscore_matrix corresponding to nodes in node_array.
        edges_times (str): 'double' or 'single' if edges should be counted twice or once.
        key (str): binary code indicating to which branch the node_array has previously belonged
        result (dict): dictionar where each node and its corresponding code (key) is stored.

    Returns:
        result (dict): final dictionary with all the leaf nodes and their respective codes.

    """

    # Initializatin of result dictionary, only necessary in the first call to the function
    if result is None:
        result = {}

    # Saves the current node array and its code in the result array
    # Result array stores all intermediate node_array lists, not only the leaf nodes
    result[key] = node_array

    # Stops if there is only one node left in the cluster (final leaf)
    if len(node_array)>1:

        # Calls the N_cut_func for the current array of nodes
        best_min_cut, best_N_cut, best_N_cut_array = N_cut_func(node_array, bitscore_matrix, edges_times)

        # Divides the nodes based on the annealer array solution
        new_nodes = assign_nodes(node_array, best_N_cut_array)
        nodes_0 = new_nodes[0]
        nodes_1 = new_nodes[1]

        # Updates bit score matrix based on the annealers array solution
        # and matches with corresponding nodes array
        bitscore_matrices = update_bit_matrix(bitscore_matrix, best_N_cut_array)
        bitscore_matrix_0 = bitscore_matrices[0]
        bitscore_matrix_1 = bitscore_matrices[1]

        # Recursevely calls itself to keep splitting the new nodes
        # Saves result in the result array each time
        split_nodes(nodes_0, bitscore_matrix_0, edges_times, key + '0', result)
        split_nodes(nodes_1, bitscore_matrix_1, edges_times, key + '1', result)

    return(result)

-------------
-------------

## Phylogenetic tree reconstruction

Now that we have our final result array, let us reconstruct the tree with the binary code we have implemented. Later, we will use the BioPython package to visualize the results, once we have them in newick format.

In [12]:
# imports all the tools related to the bioanalysis of the phylogenetic trees
from Bio import Phylo
import dendropy
from dendropy.calculate import treecompare
from itertools import combinations
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy
from io import StringIO
import matplotlib.pyplot as plt
import re
import time

In [ ]:
# PACKAGES RELATED TO R
# imports the metric "Clustering distance" directly from R
import os
os.environ['R_HOME'] = r'<path_to_R>\R\R-4.5.0'
from rpy2.robjects import r, globalenv
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import StrVector
ape = importr('ape')
treedist = importr('TreeDist')

In [14]:
def import_real_tree(filename):

    """   
    Function that takes an Excel file and imports all the trees by column.

    Args:
        filename (str): path to the file that contains the trees.

    Returns:
        row_list (list): list containing all the trees in the file in newick format.
    
    """

    # Open up the Excel file
    workbook = load_workbook(filename)
    
    # Get the first sheet
    worksheet = workbook.worksheets[0]
    
    row_list = []
    
    # Read file by rows
    for r in worksheet.rows:
        column = [cell.value for cell in r]
        row_list.append(column)
    
    return(row_list)

In [15]:
def prune(tree):

    """
    Function that takes out the intermediate node names from the newick format, leaving
    only the leaf names.

    Args:
        tree (str): tree in newick format

    Returns:
        tree_root (str): pruned tree in newick format
    """

    # substitutes string of the shape ...)NAME_OF_NODE_0001:0.1...
    # for ...):0.1... (standard format with only leave names)

    tree_nodes = re.sub(r'\)\s*[A-Za-z0-9\._]+:', '):', tree)
    tree_root = re.sub(r'\)\s*[A-Za-z0-9\._]+;', ');', tree_nodes)

    return(tree_root)

In [16]:
def dict_to_newick(tree_dict, branch_length=1.0):

    """   
    Function that transforms the reconstrcted tree dictionary in binary code (result) into newick format.

    Args:
        tree_dict (dict): dictionary containing only the leaf nodes, of the shape 
            {'00101': species_1, '10101': species_2, ...}
        branch_length (float): default branch length in case branch lengths are not provided.

    Returns:
        tree_newick (str): tree in newick format

    """

    def insert_path(tree, path, name):
        """
        Inserts a species into the nested tree dictionary following the binary path.
        
        Args:
            tree (dict): The current level of the tree.
            path (str): Binary string representing the path.
            name (str): Species name to insert.
        """
        for direction in path[:-1]:
            if direction not in tree:
                tree[direction] = {}
            tree = tree[direction]
        tree[path[-1]] = name

    def build_newick(subtree):
        """
        Recursively builds the Newick string from the nested tree dictionary.

        Args:
            subtree (dict or str): Current subtree or species name.

        Returns:
            str: Newick string for the subtree.
        """
        if isinstance(subtree, str):
            return f'{subtree}:{branch_length}'
        children = [build_newick(child) for child in subtree.values()]
        return f'({','.join(children)}):{branch_length}'

    # Build the nested tree structure from the binary paths
    root = {}
    for path, species_list in tree_dict.items():
        for species in species_list:
            insert_path(root, path, species)

    # Convert the nested tree to Newick format
    return build_newick(root) + ';'

### Percentage of correctly reconstructed tree using Robinson_Foulds distance

In order to measure the percentage of similarity between the real tree and the reconstructed one, we use the unweighted Robinson-Fould distance and the formula

$$ percentage_{correct} = \left( 1- \frac{Robinson-Foulds \ distance}{2N-6}\right)*100, $$

where $N$ denotes the total number of external tips and $2N-6$ represents the maximal possible distance two trees can take. Therefore, this formula measures how many branches are recovered in the reconstructed tree compared with the true tree.

In [17]:
def percentage_rf(tree_real, tree_reconstructed, len_nodes):

    """"
    Function that calculates the Robinson-Foulds distance as a percentage between
    two trees (real and reconstructed).

    Args:
        tree_real (str): real tree in newick format.
        tree_reconstruced (str): reconstructed tree in newick format.
        len_nodes (int): number of nodes of the tree.

    Returns:
        percentage (float): percentage of correct reconstruction.

    """

    # shared_data creates a shared taxon space of node names for the two trees to use
    shared_taxa = dendropy.TaxonNamespace()

    # read trees in newick format into the dendropy syntax
    tree1 = dendropy.Tree.get(data = tree_real, schema="newick", taxon_namespace = shared_taxa)
    tree2 = dendropy.Tree.get(data = tree_reconstructed, schema="newick", taxon_namespace = shared_taxa)

    # calculate robinson-foulds distance
    robinson_foulds_distance = treecompare.symmetric_difference(tree1, tree2)

    # implement percentage formula
    # divide by total number of possible branches
    percentage = 100 - robinson_foulds_distance*100/(2*len_nodes-6)

    return(percentage)

### Percentage of correctly reconstructed tree using Clustering distance

Sometimes, the Robinson-Foulds distance may not be an appropiate measure, as it has been critisized to yield biased results. It may not escalate well with a large number of nodes and may omit appropiate clusters if a partition close to the root is not correctly reconstructed. Thus, the Clustering Info Distace has been proposed as an alternative for the Robinson-Foulds distance in order to measure the percentage of correct reconstruction.

The Clustering Distance measures the correctly recovered clusters based on information theory metrics. It is implemented in R in the TreeDist library, so we have imported the function directly into python using the $\verb|rpy2|$ package. The function $\verb|ClusteringInfoDistace|$ can be normalized and returns a value $0$ when two trees are identical, therefore, we have implemented the percentage as

$$  percentage_{correct} = \left( 1- Clustering \ distance \right)*100. $$

In [18]:
def percentage_cd(tree_real, tree_reconstructed):

    """"
    Function that calculates the Clustering distance as a percentage between
    two trees (real and reconstructed).

    Args:
        tree_real (str): real tree in newick format.
        tree_reconstruced (str): reconstructed tree in newick format.

    Returns:
        percentage (float): percentage of correct reconstruction.

    """

    # read trees in the appropiate package format
    tree1 = ape.read_tree(text=tree_real)
    tree2 = ape.read_tree(text=tree_reconstructed)

    # calculathe the clustering distance
    dist_r = treedist.ClusteringInfoDistance(tree1, tree2, normalize=True)
    
    # make the result a float
    distance = float(dist_r[0])

    # the result indicates how dissimilar the trees are, so we calculate the inverse
    
    return((1-distance)*100)

In [19]:
def visualize(tree, file_save):

    """   
    Function to visualize a tree using matplotlib.

    Args:
        tree (str): tree to visualize in newick format.    
        file_save (str): path to output file.
    
    """

    # read tree in newick format into Phylo syntax
    handle_tree = StringIO(tree)
    tree = Phylo.read(handle_tree, 'newick')

    # build plot and save output
    fig = plt.figure(figsize=(20,18))
    axes = fig.add_subplot(1,1,1)
    Phylo.draw(tree, do_show=False, axes=axes)
    plt.savefig(file_save +'.jpg', dpi=300)
    plt.show()

In [20]:
def calculate_branch_length(tree_file):

    """   
    Function that calculates the mean branch length and standard deviation of a certain tree.

    Args:
        tree-file (str or Phylo tree): tree in newick format or already read as a Phylo tree.

    Returns:
        mean_length (float): mean branch length of the tree.
        std_length (float): standard deviation of the branch lengths of the tree.
    """

    # checks if the tree is in string format and if so, 
    # converts it to Phylo format
    if isinstance(tree_file, str) is True:
        tree = Phylo.read(StringIO(tree_file), 'newick')
    else:
        tree = tree_file

    # creates a list with all the branch lengths
    lengths = [clade.branch_length for clade in tree.find_clades() if clade.branch_length is not None]

    # ignores empty branches
    if not lengths:
        return None, None

    # calculates mean and standard deviation
    mean_length = np.mean(lengths)
    std_length = np.std(lengths)

    return mean_length, std_length

In [21]:
def write_branch_percentage_table(filename, header=None, data_rows=[]):
    """
    Writes a table with the given header and data_rows to filename.
    Each row should be a list: [filter_label, species, avg_branch_length, std_branch_length, nodes, percentage, std_percentage]
    """
    with open(filename, "a", encoding="utf-8") as f:
        # Write header
        if header:
            f.write('\t'.join(header) + '\n')
        # Write rows
        else:
            for row in data_rows:
                f.write('\t'.join(str(item) for item in row) + '\n')

## EXECUTION

In [22]:
def main(directory_of_tsv, input_file, file_real_trees):

    """   
    Main function that runs the entire process of tree reconstruction and comparison with the real tree.

    Args:
        directory_of_tsv (str): path to the directory containing the .tsv files.
        input_file (str): name of the .tsv file to process.
        file_real_trees (str): path to the excel file containing the real trees.    

    Returns:
        mean_percentage (float): mean percentage of correct reconstruction over 5 repetitions.
        std_percentage (float): standard deviation of the percentage of correct reconstruction over 5 repetitions.
        branch_length (float): mean branch length of the real tree.
        branch_std (float): standard deviation of the branch lengths of the real tree.
    """

    real_tree_lists = import_real_tree(file_real_trees)

    # applies the sequence of functions to reconstruct the tree
    nodes, bitscore_matrix = import_from_tsv(directory_of_tsv+ '\\' + input_file)       # extracts nodes and bitscore matrix
    repetitions = []
    
    # runs several repetitions for statistics
    for i in range(5):
        time_inic = time.time()
        tree_dict = split_nodes(nodes, bitscore_matrix, 'double')   # WARNING!!! quite 'mincut' en los argumentos de esta funcion           # splits the nodes with the Digital Annealer
        time_fin = time.time()
        leaves_dict = {k: v for k, v in tree_dict.items() if len(v) == 1}                # gets rid of intermediate nodes (only leaves remain)
        reconstructed_tree = dict_to_newick(leaves_dict, branch_length=1.0)              # turns dictionary into newick format

        ### REAL TREE ###

        # gets real tree with the same name from .xslx file 
        input_file_name = input_file.replace(".tsv", "")
        species = re.sub(r'^[^_]+_|_[^_]+$', '', input_file_name)
        species_list = [sp[0] for sp in real_tree_lists]
        index_tree = species_list.index(species)
        real_tree = prune(real_tree_lists[index_tree][3])                                 # intermediate nodes get pruned out

        # calculates the percentage of correct branches between the two trees
        percentage = percentage_cd(real_tree, reconstructed_tree)
        repetitions.append(percentage)

    # calculates mean and std of the percentages
    mean_percentage = sum(repetitions)/len(repetitions)
    std_percentage = np.std(repetitions)

    # calculates branch length for statistics
    branch_length, branch_std = calculate_branch_length(real_tree)

    return(mean_percentage, std_percentage, branch_length, branch_std)

In [ ]:
# specify directory of .tsv files
directory_of_tsv = r'<path_to_tsv_files>'

# specify directory of output txt and jpg
output_file = r'<path_to_output_files>'

# specify excel file of real trees in newick format
file_real_trees = r'<path_to_real_trees_excel_file>'

In [24]:
### LAUNCH ###

# set up header for the results table
header = ["species", "av_branch", "std_branch", "nodes", "av_perc", "std_perc"]
write_branch_percentage_table(output_file, header=header)

# gets list of .tsv files to explore
tsv_files = [f for f in os.listdir(directory_of_tsv) if f.endswith('.tsv')]
tsv_files_every_5 = tsv_files[::1]  # Take every 5th file for quick scan

# explores tsv files one by one
for file in tsv_files_every_5:
    input_file = os.fsdecode(file)
    if input_file.endswith(".tsv"):

        # gets species name and number of nodes for table
        species_name = re.sub(r'^[^_]+_|_[^_]+$', '', input_file)
        nodes = import_from_tsv(directory_of_tsv + '\\' + input_file)[0]

        input_file_name = input_file.replace(".tsv", "")
        real_tree_lists = import_real_tree(file_real_trees)
        # species = re.sub(r'^\d+_', '', input_file_name)
        species = re.sub(r'^[^_]+_|_[^_]+$', '', input_file_name)
        species_list = [sp[0] for sp in real_tree_lists]
        index_tree = species_list.index(species)
        newick_real = prune(real_tree_lists[index_tree][3])  

        branch_length, branch_std = calculate_branch_length(newick_real)

        if branch_length >= 1.7749:

            av_perc, std_perc, av_branch, std_branch = main(directory_of_tsv, input_file, file_real_trees)

            # saves row for the result table
            data_rows = [
                            [species_name, 
                            round(av_branch, 4),
                            round(std_branch, 4), 
                            len(nodes),
                            round(av_perc, 4),
                            round(std_perc, 4)
                            ]
                        ]
            
            # writes the row in the output file
            write_branch_percentage_table(output_file, data_rows=data_rows)

            # some prints for checking
            print('Species:', input_file)
            print('Percentage=', av_perc)
            print('Std=', std_perc)
            print('-----------------------------')



Species: 1-7750_Phy00EB5NQ_BOVIN_nd.tsv
Percentage= 56.62321117430851
Std= 0.0
-----------------------------
Species: 1-8106_Phy0002C9D_BOVIN_nd.tsv
Percentage= 37.779230043254096
Std= 0.5164254198050817
-----------------------------
Species: 1-8112_Phy00EB58T_BOVIN_nd.tsv
Percentage= 63.79204689147275
Std= 0.0
-----------------------------
Species: 1-8243_Phy003JNZY_BOVIN_nd.tsv
Percentage= 52.567579071421335
Std= 0.0
-----------------------------
Species: 1-9069_Phy00EB6SE_BOVIN_nd.tsv
Percentage= 61.14273304038684
Std= 1.0355086526991637
-----------------------------
Species: 2-0593_Phy0001YCX_BOVIN_nd.tsv
Percentage= 21.74831777882581
Std= 0.33132607494837435
-----------------------------
Species: 2-1864_Phy001QCU3_BOVIN_nd.tsv
Percentage= 63.91740814349485
Std= 2.2281587086577153
-----------------------------
Species: 2-3466_Phy00EB78E_BOVIN_nd.tsv
Percentage= 47.086966955442854
Std= 0.0
-----------------------------
Species: 2-5618_Phy00693C3_BOVIN_nd.tsv
Percentage= 42.926021112